In [1]:
from IPython.display import clear_output

from copy import deepcopy
import gymnasium as gym
import numpy as np
import pygame
import torch
from typing import Optional

from agents import Agent, HumanAgent, RLAgent
# from neural_network import

has_gpu = torch.cuda.is_available()
device = torch.device("cuda" if has_gpu else "cpu")
# Limpa texto desnecessário
clear_output()

if not has_gpu:
    print("Nenhuma GPU dedicada detectada. O treinamento do modelo pode ser muito demorado.")

In [2]:
env = gym.make("ALE/DemonAttack-v5")
print(env.observation_space)
print(env.action_space)
env.close()

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Box(0, 255, (210, 160, 3), uint8)
Discrete(6)


<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Value</p></th>
<th class="head"><p>Meaning</p></th>
<th class="head"><p>Value</p></th>
<th class="head"><p>Meaning</p></th>
<th class="head"><p>Value</p></th>
<th class="head"><p>Meaning</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">0</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">NOOP</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">1</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">FIRE</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">2</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">RIGHT</span></code></p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">3</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">LEFT</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">4</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">RIGHTFIRE</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">5</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">LEFTFIRE</span></code></p></td>
</tr>
</tbody>
</table>

In [3]:
mapping = {
    (pygame.K_a, pygame.K_d, pygame.K_SPACE): 1,    # Atirar
    (pygame.K_d, pygame.K_SPACE): 4,                # Direita e atirar
    (pygame.K_a, pygame.K_d): 0,                    # Ficar parado
    (pygame.K_a, pygame.K_SPACE): 5,                # Esquerda e atirar
    pygame.K_SPACE: 1,                            # Atirar
    pygame.K_d: 2,                                # Direita
    pygame.K_a: 3                                 # Esquerda
}

In [4]:
class Game:
    """
    Jogo representado por um ambiente do gymnasium usado para treinar um agente.
    Note que jogos com em "modo retrato", isto é, altura maior que largura, devem ser rotacionados.

    Parâmetros
    ----------
    game : str
        Nome do jogo a ser inicializado.
    rotated : bool
        Se a tela do jogo deve ser rotacionada em 90 graus.
    multiplier : int
        Multiplicador da resolução do jogo.
    human_agent : Optional[HumanAgent]
        Agente humano que irá controlar o jogo.
    """
    def __init__(self, game: str, agent: RLAgent, rotated: bool = False,
                 multiplier: int = 4, human_agent: Optional[HumanAgent] = None) -> None:
        self.env: gym.Env = gym.make(game, render_mode="rgb_array")
        self.human_agent = human_agent
        self.agent: Agent = None
        self.rl_agent = agent

        self.is_rendering: bool = False
        self.is_human_playing: bool = None
        self.screen: pygame.Surface = None
        self.clock = pygame.time.Clock()
        self.multiplier = multiplier
        self.rotated = rotated
        self.fps: int = 60

    def _load_agent(self, is_human: bool, force_reload: bool) -> None:
        """
        Carrega o agente humano ou o agente de RL, dependendo do parâmetro `is_human`.
        Para isso, uma cópia dos agentes passados no construtor é feita e os pesos,
        se existirem, são carregados. Se o agente pedido já estiver carregado, nada é feito.

        Parâmetros
        ----------
        is_human : bool
            Se o agente a ser carregado é o humano ou o de Reinforcement Learning.
        force_reload : bool
            Se o agente deve ser recarregado mesmo se já estiver carregado.
        """
        if self.is_human_playing != is_human or force_reload:
            self.is_human_playing = is_human
            self.agent = deepcopy(self.human_agent) if is_human else deepcopy(self.rl_agent)
            self.agent.load()
        elif is_human:
            self.agent.reset()

    def _create_display(self, fps: int) -> None:
        """
        Cria a janela do pygame e inicializa o jogo.

        Parâmetros
        ----------
        fps : int
            Quantidade de frames por segundo.
        """
        self.fps = fps
        if self.screen is None:
            pygame.init()
            window_size = (
                (160 * self.multiplier, 210 * self.multiplier) if self.rotated
                else (210 * self.multiplier, 160 * self.multiplier)
            )
            self.screen = pygame.display.set_mode(window_size)

    def _blackout(self) -> None:
        """Desenha um fundo preto na tela."""
        self.screen.fill((0, 0, 0))
        pygame.display.update()

    def _render(self, observation: np.ndarray) -> None:
        """Renderiza o conteúdo da observação e mantém a taxa de quadros."""
        frame = observation.repeat(self.multiplier, axis=1).repeat(self.multiplier, axis=0)
        if self.rotated:
            frame = np.flip(np.rot90(frame, k=3), axis=1)
        self.screen.blit(pygame.surfarray.make_surface(frame), (0, 0))
        pygame.display.update()
        self.clock.tick(self.fps)

    def _wait_for_input(self) -> None:
        """Fica em espera até que o usuário interaja com a janela."""
        while True:
            if pygame.event.get(pygame.QUIT) or pygame.event.get(pygame.KEYDOWN):
                return
            self.clock.tick(10)

    def train(self, render: bool = False, force_reload: bool = False, fps: int = 60) -> None:
        """
        Inicializa o jogo para treinar o agente.

        Parâmetros
        ----------
        render : bool, padrão False
            Se o jogo deve ser renderizado.
        force_reload : bool, padrão False
            Se o agente deve ser recarregado mesmo se já estiver carregado.
        fps : int
            Quantidade de frames por segundo quando o jogo é renderizado.
        """
        self.is_rendering = render
        self._create_display(fps)
        self._load_agent(is_human=False, force_reload=force_reload)
        try:
            self._train()
        except:
            self.close()
        finally:
            # Garante que os pesos do agente são salvos mesmo se o treinamento for interrompido
            self.agent.save()

    def play(self, human_player: bool = False, force_reload: bool = False, fps: int = 60) -> None:
        """
        Inicializa o jogo para ser jogado por um humano ou para testar o agente.

        Parâmetros
        ----------
        human_player : bool, padrão False
            Se o jogador será um humano interagindo com o jogo.
        force_reload : bool, padrão False
            Se o agente deve ser recarregado mesmo se já estiver carregado.
        fps : int
            Quantidade de frames por segundo.
        """
        if self.human_agent is None:
            raise RuntimeError("Não é possível jogar sem um agente humano.")
        self.is_rendering = True
        self._load_agent(is_human=human_player, force_reload=force_reload)
        self._create_display(fps)
        self._play()

    def _play(self) -> None:
        observation, info = self.env.reset()
        while True:
            if pygame.event.get(pygame.QUIT):
                self._blackout()
                return
            events = []
            for event in pygame.event.get(pygame.KEYDOWN):
                # Verifica se alguma ação especial foi escolhida
                match event.key:
                    case pygame.K_r:
                        observation, info = self.env.reset()
                    case pygame.K_TAB:
                        self._load_agent(not self.is_human_playing, force_reload=False)
                        observation, info = self.env.reset()
                    case pygame.K_ESCAPE:
                        self._blackout()
                        return
                    case _:
                        events.append(event.key)

            if self.is_human_playing:
                # Passa também as teclas que não estão mais sendo pressionadas pelo jogador
                action: int = self.agent.process_input(
                    events,
                    list(map(
                        lambda event: event.key, pygame.event.get(pygame.KEYUP)
                    ))
                )
            else:
                action: int = self.agent.choose_action(observation)
            observation, reward, terminated, truncated, info = self.env.step(action)

            if terminated or truncated:
                observation, info = self.env.reset()

            self._render(observation)

    def _train(self) -> None:
        observation, info = self.env.reset()
        while True:
            if pygame.event.get(pygame.QUIT):
                self._blackout()
                return
            for event in pygame.event.get(pygame.KEYDOWN):
                # Verifica se alguma ação especial foi escolhida
                match event.key:
                    case pygame.K_r:
                        observation, info = self.env.reset()
                    case pygame.K_TAB:
                        self.is_rendering = not self.is_rendering
                        if not self.is_rendering:
                            self.screen.fill((0, 0, 0))
                            pygame.display.update()
                        observation, info = self.env.reset()
                    case pygame.K_ESCAPE:
                        self._blackout()
                        return

            action = self.agent.choose_action(observation)
            observation, reward, terminated, truncated, info = self.env.step(action)

            if terminated or truncated:
                observation, info = self.env.reset()

            if self.rendering:
                self.render(observation)

    def close(self) -> None:
        """
        Fecha o ambiente do jogo, a janela do pygame e libera a memória do agente.
        """
        self.agent.free()
        self.env.close()
        pygame.quit()

In [5]:
# ALE/DemonAttack-v5 skips 4 frames
game = Game("DemonAttackNoFrameskip-v4", Agent(6), rotated=True, human_agent=HumanAgent(mapping))

In [6]:
game.play(human_player=True)

In [7]:
# game.train(start_rendering=True)

In [8]:
game.close()